In [0]:
from google.colab import drive
drive.mount('/content/gdrive')                                                                                                                                                                                        

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!apt-get install openjdk-8-jdk -qq > /dev/null
!pip install h2o -qq

In [0]:
!npm install -g localtunnel -qq > /dev/null
get_ipython().system_raw('lt --port 54321 >> url.txt 2>&1 &')
!cat url.txt

In [1]:
import os

print(os.getcwd())
os.chdir('gdrive/My Drive/financial-predictions')
print(os.getcwd())

C:\Users\Eduardo\Documents\1-ironhack\googleDrive\financial-predictions\colabNotebooks


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'gdrive/My Drive/financial-predictions'

In [14]:
import pandas as pd
from timeSeriesToSupervised import series_to_supervised
from transformData import minMaxScaleOnTrain
import numpy as np
import splitSequence as sq
import h2o
from h2o.automl import H2OAutoML
import ta

In [11]:
symbol = 'aep'
stock = pd.read_csv(f'../inputData/stocksLong/{symbol}.us.txt', parse_dates=True).drop(columns='OpenInt')
stock.set_index('Date', inplace=True)
display(stock.head())
stock  = ta.add_trend_ta(stock, high='High', low='Low', close='Close', fillna=True)
stock = ta.add_others_ta(stock, close='Close', fillna=True)
stock.columns
#print(stock.shape)

,Open,High,Low,Close,Volume
Date,,,,,
1970-01-02,1.5325,1.5783,1.5325,1.5631,13511
1970-01-05,1.5631,1.5783,1.5403,1.5783,23083
1970-01-06,1.5783,1.5934,1.5708,1.5708,18360
1970-01-07,1.5708,1.5783,1.5552,1.5783,10098
1970-01-08,1.5708,1.5708,1.5552,1.5552,21375


Index(['Open', 'High', 'Low', 'Close', 'Volume', 'trend_macd',
       'trend_macd_signal', 'trend_macd_diff', 'trend_ema_fast',
       'trend_ema_slow', 'trend_adx', 'trend_adx_pos', 'trend_adx_neg',
       'trend_vortex_ind_pos', 'trend_vortex_ind_neg', 'trend_vortex_ind_diff',
       'trend_trix', 'trend_mass_index', 'trend_cci', 'trend_dpo', 'trend_kst',
       'trend_kst_sig', 'trend_kst_diff', 'trend_ichimoku_a',
       'trend_ichimoku_b', 'trend_visual_ichimoku_a',
       'trend_visual_ichimoku_b', 'trend_aroon_up', 'trend_aroon_down',
       'trend_aroon_ind', 'trend_psar', 'trend_psar_up', 'trend_psar_down',
       'trend_psar_up_indicator', 'trend_psar_down_indicator', 'others_dr',
       'others_dlr', 'others_cr'],
      dtype='object')

In [12]:
columns = ['Open', 'High', 'Low', 'Volume', 'trend_macd', 'others_dr']
df_forML = series_to_supervised(list(stock['Close']), n_in=50, n_out=1, name_var='Close')
df_forML.rename(columns={'Close(t)':'y'}, inplace=True) 
for col in columns:
    df_forML = df_forML.merge(series_to_supervised(list(stock[col]), n_in=50, n_out=0, name_var=col), left_index=True, right_index=True)
df_forML = df_forML.set_index(stock.iloc[50:].index)
df_forML = df_forML.iloc[:int(len(df_forML) * 0.8)]
df_forML.head()

,Close(t-50),Close(t-49),Close(t-48),Close(t-47),Close(t-46),Close(t-45),Close(t-44),Close(t-43),Close(t-42),Close(t-41),...,others_dr(t-10),others_dr(t-9),others_dr(t-8),others_dr(t-7),others_dr(t-6),others_dr(t-5),others_dr(t-4),others_dr(t-3),others_dr(t-2),others_dr(t-1)
Date,,,,,,,,,,,,,,,,,,,,,
1970-03-16,1.5631,1.5783,1.5708,1.5783,1.5552,1.5708,1.5552,1.5097,1.5097,1.5019,...,4.449588,-0.960355,0.969667,-1.434376,-2.373368,0.000000,-0.505406,0.000000,0.000000,0.507973
1970-03-17,1.5783,1.5708,1.5783,1.5552,1.5708,1.5552,1.5097,1.5097,1.5019,1.4942,...,-0.960355,0.969667,-1.434376,-2.373368,0.000000,-0.505406,0.000000,0.000000,0.507973,0.000000
1970-03-18,1.5708,1.5783,1.5552,1.5708,1.5552,1.5097,1.5097,1.5019,1.4942,1.5248,...,0.969667,-1.434376,-2.373368,0.000000,-0.505406,0.000000,0.000000,0.507973,0.000000,-0.505406
1970-03-19,1.5783,1.5552,1.5708,1.5552,1.5097,1.5097,1.5019,1.4942,1.5248,1.4942,...,-1.434376,-2.373368,0.000000,-0.505406,0.000000,0.000000,0.507973,0.000000,-0.505406,0.000000
1970-03-20,1.5552,1.5708,1.5552,1.5097,1.5097,1.5019,1.4942,1.5248,1.4942,1.5097,...,-2.373368,0.000000,-0.505406,0.000000,0.000000,0.507973,0.000000,-0.505406,0.000000,-0.958076


In [16]:
# We scale the data based on the training period.
df_forML = minMaxScaleOnTrain(df_forML)

In [17]:
h2o.init(
    nthreads=3)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,22 mins 56 secs
H2O cluster timezone:,Europe/Paris
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,1 month and 7 days
H2O cluster name:,H2O_from_python_Eduardo_euh5m4
H2O cluster total nodes:,1
H2O cluster free memory:,6.939 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,3
H2O cluster status:,"locked, healthy"


In [18]:
dfh2o = h2o.H2OFrame(df_forML)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [19]:
train, test = dfh2o.split_frame(ratios=[0.8])

In [20]:
aml = H2OAutoML(max_models=20)
aml.train(y= 'y', training_frame=train, leaderboard_frame=test)

AutoML progress: |█
22:47:57.614: AutoML: XGBoost is not available; skipping it.

███████Failed polling AutoML progress log: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\Eduardo\\AppData\\Local\\Temp\\tmp_v0u8un4.csv'
██Failed polling AutoML progress log: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\Eduardo\\AppData\\Local\\Temp\\tmpdkj50bd0.csv'
█ (cancelled)


H2OJobCancelled: Job<$03017f00000132d4ffffffff$_ac43cd321a2a51b61521c398b63140d2> was cancelled by the user.

In [0]:
aml.leaderboard

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20200310_133143,0.301318,0.548924,0.301318,0.336726,0.0208681
GBM_2_AutoML_20200310_133143,0.30327,0.5507,0.30327,0.338034,0.0206853
GBM_grid__1_AutoML_20200310_133143_model_1,0.304954,0.552227,0.304954,0.334389,0.0220337
XGBoost_grid__1_AutoML_20200310_133143_model_3,0.30576,0.552956,0.30576,0.333477,0.0210758
GBM_3_AutoML_20200310_133143,0.305766,0.552961,0.305766,0.33999,0.0204518
XGBoost_2_AutoML_20200310_133143,0.306345,0.553485,0.306345,0.335647,0.0204553
XGBoost_1_AutoML_20200310_133143,0.306851,0.553942,0.306851,0.338911,0.0205816
GBM_4_AutoML_20200310_133143,0.307008,0.554083,0.307008,0.338656,0.0201676
StackedEnsemble_AllModels_AutoML_20200310_133143,0.307648,0.55466,0.307648,0.343002,0.0211196
GBM_1_AutoML_20200310_133143,0.310236,0.556988,0.310236,0.334538,0.0211219


In [0]:
aml.event_log

timestamp,level,stage,message,name,value
13:31:43.823,Info,Workflow,Project: AutoML_20200310_133143823,,
13:31:43.824,Info,Validation,Setting stopping tolerance adaptively based on the training frame: 0.014476040267536081,,
13:31:43.824,Info,Validation,Build control seed: -1 (random),,
13:31:43.824,Info,DataImport,training frame: Frame key: automl_training_py_3_sid_b261 cols: 251 rows: 4772 chunks: 4 size: 8696048 checksum: 2209500737200144964,,
13:31:43.824,Info,DataImport,validation frame: NULL,,
13:31:43.825,Info,DataImport,leaderboard frame: Frame key: py_4_sid_b261 cols: 251 rows: 1203 chunks: 4 size: 2243296 checksum: -6069475821277226518,,
13:31:43.825,Info,DataImport,blending frame: NULL,,
13:31:43.825,Info,DataImport,response column: y,,
13:31:43.825,Info,DataImport,fold column: null,,
13:31:43.825,Info,DataImport,weights column: null,,


In [0]:
os.getcwd()
#model_path = h2o.save_model(model=aml.leader, path="./h2o-model")
model2 = h2o.get_model(aml.leaderboard.as_data_frame()['model_id'][2])
model3 = h2o.get_model(aml.leaderboard.as_data_frame()['model_id'][3])
model4 = h2o.get_model(aml.leaderboard.as_data_frame()['model_id'][4])

In [0]:
model_path = h2o.save_model(model=model4, path="./h2o-model")
# model_path = h2o.save_model(model=model2, path="./h2o-model")
# model_path = h2o.save_model(model=model3, path="./h2o-model")

In [0]:
h2o.shutdown()